In [1]:
import pandas as pd
import numpy as np
#df_org=pd.read_csv(filepath+"application_train_merge_clear_nomissing .csv")
df_org=pd.read_csv("application_train_merge_clear_nomissing_with_ylabel_dy.csv")

In [2]:
df=df_org

In [3]:
y_=[]#找出 違約人的索引
for i in (df[df["simple_labelx"]!=0].index):
  y_.append(i)
y_1=[] #找出沒違約人的索引
for i in (df[df["simple_labelx"]==0].index):
  y_1.append(i)
df_split=df.loc[y_]
df_split0=df.loc[y_1]
df_tar1=df_split.reset_index()
df_tar0=df_split0.reset_index()
df_tar_e=df_tar0.iloc[:57886,:] #分成同等分
dfe=pd.concat([df_tar_e,df_tar1],ignore_index=True,axis=0)

In [11]:
df_X = dfe[['AMT_INCOME_TOTAL', 'DAYS_BIRTH', 'DAYS_EMPLOYED', 'DAYS_REGISTRATION',
       'DAYS_ID_PUBLISH', 'CNT_FAM_MEMBERS', 'HOUR_APPR_PROCESS_START_x',
       'OBS_60_CNT_SOCIAL_CIRCLE', 'DAYS_LAST_PHONE_CHANGE',
       'AMT_REQ_CREDIT_BUREAU_YEAR']]
df_y = dfe['simple_labelx']

In [4]:
df_X = dfe[['AMT_INCOME_TOTAL',  'DAYS_REGISTRATION',
        'CNT_FAM_MEMBERS',
       'OBS_60_CNT_SOCIAL_CIRCLE']]
df_y = dfe['simple_labelx']

In [5]:
df.head()

,Unnamed: 0,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,NAME_EDUCATION_TYPE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,FLAG_EMP_PHONE,...,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes,simple_labelx,scores
0,0,1.0,0.0,202500.0,1.0,-9461.0,-637.0,-3648.0,-2120.0,1.0,...,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,96
1,1,0.0,0.0,270000.0,3.0,-16765.0,-1188.0,-1186.0,-291.0,1.0,...,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,100
2,2,0.0,0.0,67500.0,1.0,-19046.0,-225.0,-4260.0,-2531.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100
3,3,0.0,0.0,135000.0,1.0,-19005.0,-3039.0,-9833.0,-2437.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,96
4,4,0.0,0.0,121500.0,1.0,-19932.0,-3038.0,-4311.0,-3458.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88


In [12]:
from sklearn.model_selection import train_test_split
#X , y = dfe.iloc[:,2:-2] , dfe.iloc[:,-2]
X , y = df_X  , df_y
X_train, X_test, y_train, y_test =\
    train_test_split(X, y, 
                     test_size=0.1,#超過10萬9,1分比較好 
                     #random_state=0, 
                     stratify=y)

In [7]:
 y_test

108785    1.0
37920     0.0
10023     0.0
101018    1.0
107439    1.0
         ... 
111005    1.0
29539     0.0
108453    1.0
89421     1.0
105936    1.0
Name: simple_labelx, Length: 11578, dtype: float64

## 決策樹

In [28]:
from sklearn import tree

# 建立分類器
clf = tree.DecisionTreeClassifier()
X_clf = clf.fit(X_train, y_train)

# 預測
y_test_predicted = X_clf.predict(X_test)
print(y_test_predicted)

# 標準答案
print(y_test)

[0. 1. 1. ... 1. 1. 1.]
115609    1.0
86712     1.0
81958     1.0
70090     1.0
22916     0.0
         ... 
29345     0.0
113432    1.0
38505     0.0
35850     0.0
8604      0.0
Name: simple_labelx, Length: 11578, dtype: float64


In [29]:
from sklearn import metrics
accuracy = metrics.accuracy_score(y_test, y_test_predicted)
print(accuracy)

0.5251338745897391


In [30]:
df.scores

0          96
1         100
2         100
3          96
4          88
         ... 
307506     96
307507    100
307508     90
307509     96
307510     64
Name: scores, Length: 307511, dtype: int64

## Random Forest

In [13]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=200,max_depth=10, random_state=0,criterion='entropy')
clf.fit(X_train, y_train)



RandomForestClassifier(criterion='entropy', max_depth=10, n_estimators=200,
                       random_state=0)

In [14]:
clf.feature_importances_

array([0.04877946, 0.1193811 , 0.06798393, 0.06774107, 0.07394283,
       0.01502251, 0.03414546, 0.02430674, 0.38207739, 0.1666195 ])

In [15]:
print('訓練集: ',clf.score(X_train,y_train))
print('測試集: ',clf.score(X_test,y_test))

訓練集:  0.6353436858168416
測試集:  0.6126273967870098


In [16]:
print('分類的預測結果：')
pred = clf.predict(X_test) 
print(pred) #觀察預測結果AA
print('真實數據：')
print(y_test.values)  #觀察真實數據(Test data)

分類的預測結果：
[1. 1. 0. ... 0. 0. 1.]
真實數據：
[1. 1. 0. ... 0. 1. 1.]


In [143]:
pred

array([1., 1., 1., ..., 0., 1., 1.])

In [144]:
len(pred)

11578

In [145]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test,pred)

array([[3325, 2464],
       [2169, 3620]])

In [20]:
import joblib

In [21]:
joblib.dump(clf,'clf.pkl')

['clf.pkl']